# Predict

In [324]:
def generate_input(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0, opponent_hand_cards=None):
    obs = [np.zeros([4, 8, 3]), np.zeros([4])]
    
    for card in hand_cards:
        obs[0][card.color.value][card.value.value][0] = 1
        
    if table_card is not None:
        obs[0][table_card.color.value][table_card.value.value][1] = 1
        
    if opponent_hand_cards is not None:
        for card in opponent_hand_cards:
            obs[0][card.color.value][card.value.value][2] = 1

    obs[1][0] = (own_tricks == 1 or own_tricks == 3)
    obs[1][1] = (own_tricks == 2 or own_tricks == 3)

    obs[1][2] = (opponent_tricks == 1 or opponent_tricks == 3)
    obs[1][3] = (opponent_tricks == 2 or opponent_tricks == 3)
    
    return obs

In [325]:
def card(color, value):
    for card in env.cards:
        if card.value == value and card.color == color:
            return card
        
    return None

In [326]:
def calc_correct_output_sample(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0, opponent_hand_cards=None):
    global sample_outputs
    env.reset()
    env.cards_left = env.cards[:]
    random.shuffle(env.cards_left)
    
    for card in hand_cards:
        env.cards_left.remove(card)
    env.players[0].hand_cards = hand_cards[:]
        
    if opponent_hand_cards is None:
        env.players[1].hand_cards = []
        for i in range(len(hand_cards) - (1 if table_card is not None else 0)):
            env.players[1].hand_cards.append(env.cards_left.pop())      
    else:
        env.players[1].hand_cards = opponent_hand_cards[:]
    
    env.players[0].tricks = own_tricks
    env.players[1].tricks = opponent_tricks
    env.table_card = table_card
    
    #root = State(1, env.get_state(), env.current_player)
   # a, p = mcts_game_step(root, 100)
    #env.set_state(root.env_state)
    #print(p)
    #draw_tree(root)
    #print("")
    #draw_path(root)
    #obs, rew, is_done, _ = env.step(hand_cards[0].id)
   # print(rew, is_done)

    #env.render('human')
    #root = State(1, env.get_state())
    #a, p = mcts_game_step(root, 1000, False)
    #draw_tree(root)
    #print("")
    #draw_tree(root.childs[0])
    
    #draw_path(root)
    #reset_samples(1)
    p = search(env.regenerate_obs())[1]
    #postprocess_samples()
        
    return p#sample_outputs[0]

In [327]:
def calc_correct_output(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):    
    correct_output = None
    for i in range(100):
        sample_outputs = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks)
        if correct_output is None:
            correct_output = sample_outputs
        else:
            correct_output += sample_outputs
    return correct_output / 100

In [328]:
def predict(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
    output = model.predict_single(input)[0]
    correct_output = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks)
    print(output,correct_output)
    id = 0
    for card in hand_cards:
        print(str(card.color) + " " + str(card.value) + " => " + str(output[card.id]) + " (" + str(correct_output[id]) + ")")
        id += 1

In [329]:
def create_eval_cache():
    eval_cache_input = []
    eval_cache_output = []
    for i in range(1000):
        own_tricks = random.randint(0, 2)
        opponent_tricks = random.randint(0, 2)
        table_card_enabled = random.randint(0, 1)
        hand_cards = random.sample(env.cards, (5 - own_tricks - opponent_tricks) * 2 + table_card_enabled)
        table_card = hand_cards.pop() if table_card_enabled else None
        
        opponent_hand_cards = []
        for i in range(len(hand_cards) // 2):
            opponent_hand_cards.append(hand_cards.pop())

        eval_cache_input.append(generate_input(hand_cards, table_card, own_tricks, opponent_tricks, opponent_hand_cards))
        output = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks, opponent_hand_cards)
        
        outputs_per_card = np.zeros((32,))
        for i in range(len(hand_cards)):
            outputs_per_card[hand_cards[i].id] = output[i]
            
        eval_cache_output.append(outputs_per_card)
        
    return eval_cache_input, eval_cache_output
        
def eval(model, eval_cache_input, eval_cache_output):
    correct = 0
    for i in range(len(eval_cache_input)):

        output = model.predict_single(eval_cache_input[i])[0]
        correct_output = eval_cache_output[i]
                    
        correct += correct_output[np.argmax(output)] == correct_output[np.argmax(correct_output)]
        #if correct_output[np.argmax(output)] != correct_output[np.argmax(correct_output)]:
        #    print(correct_output[np.argmax(output)], correct_output[np.argmax(correct_output)])
            
    return correct / len(eval_cache_input)


In [42]:
eval_cache_input, eval_cache_output = create_eval_cache()

In [18]:
with open('eval_cache', 'wb') as fp:
    pickle.dump([eval_cache_input, eval_cache_output], fp)

In [330]:
with open ('eval_cache', 'rb') as fp:
    eval_cache_input, eval_cache_output = pickle.load(fp)

In [43]:
eval(model, eval_cache_input, eval_cache_output)

0.979

In [67]:
predict([
    card(Color.EICHEL, Value.ACHT), 
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SAU)
    ], None, 0, 2)


[0.29292929 0.29292929 0.41414141]


FigureCanvasNbAgg()

[2.3766297e-03 2.8347185e-01 8.2801534e-03 4.5855572e-03 2.1792692e-03
 6.9862767e-03 2.8642794e-01 1.3888585e-03 3.8995058e-03 1.2016824e-03
 3.0918578e-03 1.1405852e-03 3.3063155e-03 1.5768711e-03 2.5065003e-03
 1.5009379e-03 4.8645613e-01 2.3454806e-04 1.3556489e-03 1.6275608e-03
 1.1519113e-03 7.1208691e-04 1.4781591e-04 1.1066282e-03 2.2558696e-03
 3.8601116e-03 1.0180498e-02 1.9784123e-03 2.0697922e-03 3.8397033e-03
 3.1332180e-03 6.4104568e-04] [0. 0. 0.]
Color.EICHEL Value.ACHT => 0.28642794 (0.0)
Color.EICHEL Value.KOENIG => 0.28347185 (0.0)
Color.HERZ Value.SAU => 0.48645613 (0.0)


In [371]:
predict([
    card(Color.EICHEL, Value.UNTER), 
    card(Color.GRUEN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SIEBEN),
    card(Color.GRUEN, Value.ZEHN),
    ],card(Color.GRUEN, Value.OBER), 0, 0)

ValueError: Error when checking : expected input_9 to have shape (4, 8, 2) but got array with shape (4, 8, 3)

In [68]:
predict([
    card(Color.HERZ, Value.SIEBEN), 
    card(Color.GRUEN, Value.KOENIG),
    card(Color.HERZ, Value.ACHT),
    card(Color.EICHEL, Value.SAU),
    card(Color.GRUEN, Value.UNTER),
    ],card(Color.SCHELLN, Value.OBER), 0, 0)

[0.56565657 0.05050505 0.29292929 0.03030303 0.06060606]


FigureCanvasNbAgg()

[1.10929282e-02 1.56451424e-04 9.00905870e-05 1.03208557e-04
 4.59249655e-04 6.79244171e-04 5.41189947e-05 5.84263878e-04
 1.26359097e-04 5.17860539e-02 4.95364584e-05 3.42544354e-02
 2.93370598e-04 1.12345464e-04 1.43227502e-04 1.80945266e-04
 2.85971822e-04 1.01780643e-04 5.51378645e-04 1.82552583e-04
 2.16027140e-03 5.32464292e-05 5.05821109e-01 6.87462747e-01
 3.92161019e-04 1.10231026e-03 8.13166043e-05 5.29427853e-05
 1.98504014e-04 5.72398058e-05 1.37993964e-04 2.24768432e-04] [1. 1. 1. 1. 1.]
Color.HERZ Value.SIEBEN => 0.68746275 (1.0)
Color.GRUEN Value.KOENIG => 0.051786054 (1.0)
Color.HERZ Value.ACHT => 0.5058211 (1.0)
Color.EICHEL Value.SAU => 0.011092928 (1.0)
Color.GRUEN Value.UNTER => 0.034254435 (1.0)


In [ ]:
hard_input = generate_input([
    card(Color.EICHEL, Value.UNTER), 
    #card(Color.GRUEN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SIEBEN),
    card(Color.GRUEN, Value.ZEHN),
    ],None, 1, 0)
model.predict_single(hard_input)

In [ ]:
for sample_input in sample_inputs[0]:
    if np.array_equal(sample_input,hard_input[0]):
        print("test")

In [80]:
compare(model, None)

0.48

In [72]:
match([model, None], True)

[0. 0. 1. 0. 0.]
[1. 1. 0. 1.]
[1. 1. 0.]
[1. 0.]
[1.]


1